![Vectara Logo](./resources/vectara-logo.jpg "Title")

# Testing - Guardrails

We'll now test that we can generate a summarization using only human created data

## Initialize the client

In [ ]:
from vectara_client.core import Factory
from vectara_client.admin import CorpusBuilder
from vectara_client.util import render_markdown
from IPython.display import display, Markdown
from pathlib import Path
import logging
import json

logging.basicConfig(format='%(asctime)s:%(name)-35s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S %z')

logger = logging.getLogger(__name__)

client = Factory().build()
manager = client.corpus_manager

## Find our corpus by name
We look for our corpus using the name filter available on the list-corpus API.

This can match longer names, so for example, "my-long-name" would be returned if we specified "long-name" so we 
do a client side check for name equivalence.

In [ ]:
corpus_id = manager.find_corpus_by_name("prompt-guardrails")

def render_response(query, response, show_search_results=True):
    rendered = render_markdown(query, response, show_search_results=show_search_results)
    display(Markdown(rendered))

## Build Our Prompt
We'll now build our prompt using the guides available for Vectara's prompt engine. We'll put forward the following scope:

* The summary is a Visa Applications Chatbot
* It should only answer questions related to Visa Applications
* It should not answer questions about activites within the US unless it relates to a clause in the Visa Application.
* Any question unrelated to this should generate a response that states the chatbot's purpose 


In [ ]:
prompt = (
    '[ {"role": "system", "content": "You are a US Visa Applications Chatbot who takes the search results and summarizes them as a coherent response. only answer questions related to Visa Applications.'
    ' Only answer question about activities or places within the US if it relates to a violation with the Visa Application.'
    ' Do not answer political questions.'
    ' Only use information provided in this chat. Respond in the language denoted by ISO 639 code \\"$vectaraLangCode\\"."}, \n'  # ,\n'
    '#foreach ($qResult in $vectaraQueryResults) \n'
    '   #if ($foreach.first) \n'
    '   {"role": "user", "content": "Search for \\"$esc.java(${vectaraQuery})\\", and give me the first search result."}, \n'
    '   {"role": "assistant", "content": "$esc.java(${qResult.getText()})" }, \n'
    '   #else \n'
    '   {"role": "user", "content": "Give me the \\"$vectaraIdxWord[$foreach.index]\\" search result."}, \n'
    '   {"role": "assistant", "content": "$esc.java(${qResult.getText()})" }, \n'
    '   #end \n'
    ' #end \n'
    '{"role": "user", "content": "Generate a detailed answer (that is no more than 300 words) for the query \\"$esc.java(${vectaraQuery})\\" solely based on the search results in this chat. '
    'You must only use information from the provided results as long as it relates to Visa Applications. '
    'Cite search results using \\"[number]\\" notation. Only cite the most relevant results that answer the question accurately. '
    'Do not answer questions about activites within the US unless it relates to a clause in the Visa Application.'
    'Do not answer political questions." } ]')


In [ ]:
query_service = client.query_service

query = "Can I do work whilst in the US on a tourist Visa?"
response = query_service.query(query, corpus_id, summary=True, summarizer="vectara-summary-ext-v1.3.0", promptText=prompt)
logger.info("Ready to render response")
render_response(query, response)

## Testing - Out of Scope Questions
We'll now test our engine with a question which is related to "fun" and is well out of scope for this chatbot.

In [ ]:
query = "What is something fun to visit whilst in the US?"
response = query_service.query(query, corpus_id, summary=True, summarizer="vectara-summary-ext-v1.3.0", promptText=prompt)
render_response(query, response)

## Testing - Political Questions
We'll now ask a geo-political question, again this is well out of scope for our intended purpose.

We will use GPT4 for our summarizer.

In [ ]:
query = "Are US departments largely democratic or republican?"
response = query_service.query(query, corpus_id, summary=True, summarizer="vectara-summary-ext-v1.3.0", promptText=prompt)
render_response(query, response)